In [1]:
%matplotlib inline
import sys
import math
import time
import random
import collections
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from Polymer import Polymer
from copy import deepcopy
import scipy.io as sio
import os.path

In [ ]:
def recursiveGrowth(pol):
    global polarray
    global max_polymer_size
    global polymer_count
    if (len(polarray) >= polymer_count):
        return
    weight = pol.addBead()
    avweight = getAverageWeight();
    upLim = 2 * avweight / pol.weight3
    lowLim = 1.2 * avweight / pol.weight3
    if (pol.size >= max_polymer_size):
        return
    if (pol.weight > upLim):
        pol.weight = pol.weight / 2;
        tpol = deepcopy(pol)
        polarray.append(tpol)
        if (len(polarray)%1000 == 0):
            print("Simulation progress: " + str(int(len(polarray)*100/polymer_count)) + "%") 
            sys.stdout.flush()
        recursiveGrowth(pol)
        recursiveGrowth(tpol)
    if (pol.weight < lowLim):
        rand = random.uniform(0,1)
        if (rand < 0.5):
            pol.weight = pol.weight * 2
            recursiveGrowth(pol)
        else:
            pol.alive = False;

def getAverageWeight():
    global polarray;
    weight_count = 0
    weight = 0
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            weight_count = weight_count + 1
            weight += polarray[i].weight
    avgweight = weight / weight_count
    return avgweight

def Grow(polarray):  # Add a bead to all living polymers
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            polarray[i].addBead()
    return polarray

def Prune(polarray):  # Prune the polymers on their average weight
    # Calculate the average weight of the living polymers
    weight = 0
    alive = 0
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            weight += polarray[i].weight
            alive += 1
    weight = weight / alive;
    # Prune the polymers on their weight
    upLim = 2 ** (math.log(alive,10)-1)
    lowLim = 1.5 ** (math.log(alive,10)-1)
    print("Alive: "+str(alive) + " Weight: "+str(weight))
    print("Uplim = "+str(upLim)+" LowLim = "+str(lowLim))
    for i in range(0, len(polarray)):
        if (polarray[i].alive):
            if (polarray[i].weight > upLim * weight / polarray[i].weight3): # Branch
                polarray[i].weight /= 2;
                polarray.append(deepcopy(polarray[i]))
            if (polarray[i].weight < lowLim * weight / polarray[i].weight3): # Prune
                rand = random.uniform(0,1)
                if (rand < 0.5):
                    polarray[i].weight *= 2
                else:
                    polarray[i].alive = False;
    return polarray

In [ ]:
bead_position = np.zeros([2,2])       # Initialize array
bead_position[1,:] = [1,0]            # Set initial condition
polymer_size = 2                      # Amount of beads in the polymer
max_polymer_size = 250                # Maximum polymer size
polymer_count = 1E5                   # Amount of polymers that we are simulating

T = 5                                 # Temperature in kelvin
epsilon = 0.25                        # Epsilon voor de lennard Jones
sigma = 0.8                           # Sigma voor de Lennard Jones

theta = 6                             # Number of angles
theta_weight = np.zeros([theta,1])    # Weight of angles
theta_weight_sum = 0                  # Sum of the weights
polymer_weight = 1                    # Weight

polarray = []                         # List containing all the polymers

# We start with 5 polymers.
for i in range(0,1000):
    pol = Polymer(bead_position, polymer_weight, 2, theta, epsilon, sigma, T)
    polarray.append(pol)

for l in range (2,250):
    polarray = Grow(polarray)
    polarray = Prune(polarray)
    print("Polymer length " + str(len(polarray)) + "  Size = "+str(l)) 
    sys.stdout.flush()
print("Done")

Alive: 1000 Weight: [ 4.85693667]
Uplim = 3.9999999999999987 LowLim = 2.2499999999999996
Polymer length 1716  Size = 2
Alive: 1347 Weight: [ 16.08155037]
Uplim = 4.37525649426263 LowLim = 2.371171797784095
Polymer length 2344  Size = 3
Alive: 1588 Weight: [ 61.61206277]
Uplim = 4.597502942812151 LowLim = 2.44090307891323
Polymer length 2771  Size = 4
Alive: 1914 Weight: [ 220.29136016]
Uplim = 4.863321386566917 LowLim = 2.522493339331174
Polymer length 3109  Size = 5
Alive: 2133 Weight: [ 847.11455163]
Uplim = 5.024537676343404 LowLim = 2.5710761045362913
Polymer length 3470  Size = 6
Alive: 2355 Weight: [ 3247.49838258]
Uplim = 5.176550007155595 LowLim = 2.616295932781127
Polymer length 3843  Size = 7
Alive: 2579 Weight: [ 12616.24379519]
Uplim = 5.320092544901394 LowLim = 2.6584928889879937
Polymer length 4168  Size = 8
Alive: 2725 Weight: [ 50357.59027798]
Uplim = 5.409017280697555 LowLim = 2.684397059468935
Polymer length 4480  Size = 9
Alive: 2862 Weight: [ 201756.95688087]
Uplim 

In [ ]:
print(len(polarray))
dist = np.zeros([max_polymer_size+1, 3])
positions = np.zeros([max_polymer_size+1, len(polarray),2])
for i in range(2, max_polymer_size+1):
    dist[i,0] = i;
    dis = 0
    c = 0
    for j in range(0, len(polarray)):
        if (polarray[j].size >=i):
            dis += polarray[j].getLength(i)
            positions[i,j,:] = polarray[j].getPosition(i-1)
            c = c + 1
    if (c > 0):
        dist[i,1] = (dis / c)
    else:
        dist[i,1] = 0
    dist[i,2] = dist[i,0] ** 0.75
name_i = 0;
while (os.path.isfile('DataPoints/data_file_'+str(name_i)+'.mat') != False):
    name_i = name_i + 1
sio.savemat('DataPoints/data_file_'+str(name_i)+'.mat', {'Distances':dist, 'Positions':positions})